<a href="https://colab.research.google.com/github/JanEggers-hr/chatgpt-playground/blob/main/TalkToPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TalkToPDF (BETA)

*v1.0* - Ein Notebook, mit dem man (mit einem gespendeten API-Token) chatGPT nutzen kann, um einen Text zu analysieren - gewissermaßen mit dem Text zu chatten. 

Das PDF wird mit einer Standard-Bibliothek vergleichsweise stumpf eingelesen; die Textlänge muss unter der Grenze bleiben, die das
Token-Limit des jeweiligen Modells vorgibt - das sind schätzungsweise 2-3 Seiten Text bei GPT-3.5, 5 Seiten bei GPT-4. Das deutlich größere GPT4-32k steht derzeit noch nicht zur Verfügung. 

Aufpassen; das Chatten kann ins Geld gehen! Das Modell hat kein Gedächtnis - man muss den Text bei jeder Anfrage komplett mit übertragen. 

Es wird empfohlen, eine Browser-Erweiterung wie den [Colab Automatic Clicker](https://addons.mozilla.org/en-US/firefox/addon/colab-automatic-clicker/) oder [Colab Auto Reconnect (Chrome)](https://chrome.google.com/webstore/detail/colab-auto-reconnect/ifilpgffgdbhafnaebocnofaehicbkem) zu nutzen, damit die Colab-Session offen bleibt. 

## Vorbereitungen

*Bitte einmal kurz auf die nächste Zelle klicken, um den Vorbereitungs-Code auszuführen: Einstell-Widgets erzeugen, die Library für die OpenAI-API laden.*

In [ ]:
#@title
import requests
import json
import math

# ipywidgets ist schon installiert
import ipywidgets as widgets
from IPython.display import display

# Modelle und Kosten definieren
# Kosten in US-Dollar je 1000 Tokens
models_token_info = {'gpt-3.5-turbo': {
                                        'pricing': 0.002,
                                        'max_tokens': 4096
                                      },
          'gpt-4': {
                                        'pricing': 0.03,
                                        'max_tokens': 8192
                                      },
          'gpt-4-32k': {
                                        'pricing': 0.06,
                                        'max_tokens': 32768
                                      }}

textbox_max_tokens = widgets.Text(
    value='0',
    placeholder='0',
    description='Max. Token:',
)

area_system = widgets.Textarea(
    value = 'Du bist chatGPT, ein KI-Sprachsystem. Du bist freundlich \
und hilfsbereit und löst alle Aufgaben Schritt für Schritt.\n',
    rows=10,
    description = 'System:'
)

# Temperatur-Slider
slider_temperature = widgets.FloatSlider(
    value=0.2,
    min=0,
    max=1,
    step=0.1,
    description='Temperatur:',
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

# Best-of-Slider
slider_bestof = widgets.IntSlider(
    value=1,
    min=1,
    max=4,
    description='Best Of:',
    orientation='horizontal',
    readout=True
)


dropdown_model = widgets.Dropdown(
    # Nimm die oben definierte Preisliste als Basis
    options=list(models_token_info.keys()),
    value=list(models_token_info.keys())[1], # gpt-4 als Default
    description='Modell:',
)

textbox_stop = widgets.Text(
    value='###\n',
    placeholder="###",
    description="Stop-Token:"
)

html_warning = widgets.HTML(description = "Status:", value = "OK")

# Bisschen breiter anzeigen
textbox_max_tokens.layout.width = '200px'
dropdown_model.layout.width = '300px'
area_system.layout.width = '600px'

# Vorbereitungen für die Einstellungen sind getan - jetzt die OpenAI-Libraries
print("Widgets eingerichtet.")

# Tokenizer Tiktoken einbinden
!pip install -q tiktoken
import tiktoken
print("Tokenizer tiktoken geladen.")

# Hilfsfunktion: Token berechnen
def num_tokens_from_string(string: str) -> int:
    """Returns the number of tokens in a text string."""
    # cl100k_base ist der Tokenizer für Davinci, GPT-3 und GPT-4
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

# OpenAI-API-Library einbinden
!pip install -q openai
import openai
print("OpenAI-API-Library geladen.")

# https://github.com/py-pdf/benchmarks
!pip install -q pypdfium2
import pypdfium2 as pdfium
print("PDF-Importer pypdfium geladen.")

# Funktion wird bei Veränderung ausgeführt
def update_params(change):
    global temperature
    global max_tokens
    global system_prompt
    global system_tokens
    global model
    global stoptokens
    global best_of
    temperature = slider_temperature.value
    model = dropdown_model.value
    best_of = slider_bestof.value
    # Token-Obergrenze umrechnen
    try:
        max_tokens = int(textbox_max_tokens.value)
        if max_tokens == 0:
            max_tokens = None
        else: 
            limit = model_token_info.get(model)['max_tokens']
            if max_tokens + system_tokens > limit: 
                html_warning = "<strong><em>Token-Obergrenze des Modells überschritten</em></strong>"
            else:
                html_warning = "OK"
    except ValueError:
        max_tokens = None
    textbox_max_tokens.value = f'{max_tokens}'
    system_prompt = area_system.value
    system_tokens = num_tokens_from_string(system_prompt)
    stoptokens = textbox_stop.value
    if (stoptokens == ""):
      stoptokens = None

# 1x aufrufen.
update_params(0)

# Verbinde die Widgets mit der Funktion zur Verarbeitung der Werte
textbox_max_tokens.observe(update_params, 'value')
slider_temperature.observe(update_params, 'value')
slider_bestof.observe(update_params, 'value')
area_system.observe(update_params, 'value')
textbox_stop.observe(update_params, 'value')
dropdown_model.observe(update_params, 'value')

## Einstellungen für das Modell

Basiseinstellungen: Temperatur (tendenziell niedriger als sonst, um reproduzierbare Ergebnisse zu bekommen), Modell, Stop-Token (der Text des PDF wird in Stop-Token eingefasst, um dem Modell eine Sinneinheit zu signalisieren), und das System-Prompt (die Aufgabenbeschreibung). 

In [ ]:
#@title
from getpass import getpass
openai.api_key = getpass("OpenAI-API-Key eingeben: ")
codeblock = False       # Hat Ausgabe eines Codeblocks begonnen?

def on_chatbot_reset_clicked(button):
    global previous_messages
    global spent_tokens
    global spent_dollars
    previous_messages = []
    spent_tokens = 0
    spent_dollars = 0.00
    chatbot_output_area.value = ''

# Die Einstellungs-Widgets anzeigen
# Setzt die globalen Variablen temperature, system_prompt, api_key, model, stoptokens
display(slider_temperature,
#        slider_bestof,
        dropdown_model,
        textbox_stop, 
        textbox_max_tokens,
        area_system)

# Hilfsfunktion : Kosten berechnen. 
def pricing(tokens):
    price = models_token_info.get(model)['pricing']
    # Kosten in Dollar zurückgeben
    return(tokens * price / 1000)

def chatbot(prompts):
    # Prompt 
    response = openai.ChatCompletion.create(
        model=model,
        messages=prompts,
        n=1,
#        best_of = best_of,
        stop=stoptokens,
        max_tokens=max_tokens,
        temperature=temperature,
        stream = True
    )
    return response

text_tokens = widgets.HTML(
    value = '<b>Verbrauchte Token</b>: 0 ($0.00)'
)

# Define the widget for displaying token usage
def update_token_usage_widget(value):
    global spent_tokens
    global spent_dollars
    spent_tokens += value
    spent_dollars += pricing(value)
    token_usage_text = f'PDF: {number_tokens} Token ($ {pricing(number_tokens):.3f}) <br>\
<b>Verbrauchte Token:</b> Diesmal {value} ($ {pricing(value):.3f}), \
insgesamt {spent_tokens} ($ {spent_dollars:.3f}) '
    text_tokens.value = token_usage_text

chatbot_output_area = widgets.HTML(
    value='',
    description='Dialog:',
    layout=widgets.Layout(width='100%')
)
# Hilfsfunktion: Token berechnen
def calculate_tokens(string: str) -> int:
    """Returns the number of tokens in a text string."""
    # cl100k_base ist der Tokenizer für Davinci, GPT-3 und GPT-4
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

# Ausgaben von GPT formatieren:
# - \n in <br> umsetzen
# - Codeblöcke mit <pre><code> beginnen und abschließen

import re

def gptparse(text):
    global codeblock
    text = re.sub('\r?\n','<br>',text)
    ### Codeblock-Zeichenfolge ``` gefunden?
    if re.search('\`\`\`',text):
        if codeblock:
            codeblock = False
            text = re.sub('\`\`\`','</pre></code>',text)
        else: 
            codeblock = True
            text = re.sub('\`\`\`','<code><pre>',text)
    return text


chatbot_output_area = widgets.HTML(
    value='',
    description='Dialog:',
    layout=widgets.Layout(width='100%')
)

# Define the function to be called when the chatbot is used
def on_chatbot_button_clicked(button):
    global chatbot_output
    # Get the user's input and display it
    user_input = user_text.value
    user_text.value = ''
    # Generate a response from the chatbot
    chatbot_output_area.value += f'<p style="font-family: courier;"><b>Du</b>: {user_input}</p>'
    messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": pdf_prompt},
            *previous_messages,
            {"role": "user", "content": user_input},
        ]
    chatbot_output_area.value += '<p style="font-family: courier; font-style: italic;"><b>Chatbot: </b>' 
    # Stream-Objekt mit der Antwort
    chatbot_response = chatbot(messages)
    collected_messages = []   # braucht man nicht zwingend
    # Anzahl von Tokens mit der User-Frage initiieren
    chunk_tokens = calculate_tokens(user_input)
    # Iteriere über die Chunks (die Brocken )
    for chunk in chatbot_response:
        chunk_message = chunk['choices'][0]['delta']  # extract the message
        collected_messages.append(chunk_message)  # save the event response
        # Ausgabefenster: Neuen Chunk anhängen
        chatbot_output = gptparse(chunk_message.get('content', ''))
        chatbot_output_area.value += chatbot_output
        update_token_usage_widget(calculate_tokens(chatbot_output))
    # Stream-HTML-Block abschließen
    chatbot_output_area.value += '</p>'
    # Antwort komplett in die Chathistorie aufnehmen
    chatbot_output = ''.join([m.get('content', '') for m in collected_messages])
    previous_messages.extend([
        {"role": "user", "content": user_input},
        {"role": "assistant", "content": chatbot_output},
    ])
    

# Define the chatbot input and output widgets
user_text = widgets.Text(
    placeholder='...',
    description='Du:',
    layout=widgets.Layout(width='60%'),
)

# Definiere den Absenden-Button und binde ihn an on_chatbot_button_clicked
chatbot_button = widgets.Button(
    description='Absenden',
    layout=widgets.Layout(width='15%'),
)
chatbot_reset = widgets.Button(
    description = 'Reset',
    layout=widgets.Layout(width='15%')
)
chatbot_button.on_click(on_chatbot_button_clicked)
chatbot_reset.on_click(on_chatbot_reset_clicked)
# Abschicken auch durch Return in der user_text Box
user_text.on_submit(on_chatbot_button_clicked)



## PDF einlesen, an den Chatbot übergeben, Eingaben erwarten

Nach der Eingabe des OpenAI-API-Tokens lädt das Skript ein PDF hoch, extrahiert den Text, und ermöglicht eine Konversation darüber. 

In [ ]:
#@title
previous_messages = []
spent_tokens = 0
spent_dollars = 0.00
area_system.value = "Du beantwortetst Fragen zum Text. Du erklärst, was im Text \
zu finden ist und was in den Trainingsdaten des Sprachmodells zu finden ist."
system_prompt = area_system.value
system_tokens = num_tokens_from_string(system_prompt)

from google.colab import files

def import_pdf():
  global pdf_prompt
  file_upload = files.upload()
  # Namen des hochgeladenen PDF extrahieren
  pdf_name = str(list(file_upload.keys())[0])
  pdf = pdfium.PdfDocument(file_upload[pdf_name])
  # n_pages = len(pdf)
  # Das ist erst mal ganz stumpf: Iteriere durch alle Seiten und hänge sie 
  # an einen großen Python-String. 
  #
  # Verfeinerte Strukturierung - Seitenzahlen oä - später. 
  text_all = ""
  for page_num,page in enumerate(pdf):
      text_all += f"\n\nSEITE: {page_num+1}\n\n"
      # Load a text page helper
      textpage = page.get_textpage()
      # Extract text from the whole page
      text_all += textpage.get_text_range()
  # Update Kosten
  return(text_all)

text_all = import_pdf()
number_tokens = num_tokens_from_string(text_all)
print(f"{len(text_all)} Zeichen - in Token: {number_tokens}")
pdf_prompt = "###\nAnalysiere diesen Text:\n" + text_all + "\n###\n"

# Die eigentliche Chatbot-Funktion findet sich in der Funktion
# on_chatbot_button_clicked()

# Display the chatbot widgets
display(html_warning)
display(text_tokens)
display(chatbot_output_area, user_text, chatbot_button, chatbot_reset)


## Bekannte Probleme

- Manchmal verschluckt das Eingabefeld die letzten 1, 2 Zeichen
- Reset-Button funktioniert nicht (sollte Neuladen triggern)
- Obergrenze des Modells werden derzeit nicht angezeigt

## Nice-to-have: Verbesserungen

- Inhaltsverzeichnis des PDF generieren und ebenfalls übergeben
- Einen "Zusammenfassungsmodus" bauen: Jede Seite wird vom Modell auf einen Digest eingedampft. Das Modell bekommt dann die Liste aller Digests - und die Anweisung, den Volltext von relevanten Seiten über ```GETPAGE n``` anzufordern. (Ggf. durch Stoptoken trennen.) Das ermöglicht PDFs bis zu mehreren Dutzend Seiten auszuwerten. 
- Codeblöcke korrekt formatieren